In [68]:
import itertools as it
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import functools
import math
import fractions
from scipy.interpolate import lagrange
from numpy import linalg as LA

In [59]:
def KronDel(x, y):
    if x == y:
        return 1.
    else:
        return 0.

In [60]:
def CanonicalMPI(func, x, k, say = 0):
    for i in range(k):
        if(say != 0):
            print (i, "th iteration, x is ", x)
        x = func(x)
    if(say != 0):
            print (k, "th iteration, x is ", x)
    return x

$\textbf{Метод отрезков}$

In [21]:
#g - функция выбора - пара точек на вход, пара на выходе
def sections(g, f, a, b, k, say = 0):
    I = np.array([a, b])
    for i in range(k):
        if(say != 0):
            print(i, "th iteration, x1 = ", I[0], ", x2 = ", I[1], " |x1 - x2| = ", np.abs(I[0] - I[1]))
        c = g(I)[0]
        d = g(I)[1]
        if (f(c) > f(d)):
            I = np.array([c, b])
            a = c
        else:
            I = np.array([a, d])
            b = d
    if(say != 0):
            print(k, "th iteration, x1 = ", I[0], ", x2 = ", I[1], " |x1 - x2| = ", np.abs(I[0] - I[1]))
    return I[0]

In [50]:
#Пример:
def g(I):
    return np.array([I[0] + (1/3)*(I[1] - I[0]), I[0] + (2/3)*(I[1] - I[0])])
def f1(x):
    return x**2
sections(g, f1, -1., 1., 25, 0)

-3.4143342277530024e-05

$\textbf{Метод золотого сечения}$

In [32]:
def coolg(I):
    return np.array([I[0] + (I[1] - I[0])*((3-(5**0.5))/2), I[1] - (I[1] - I[0])*((3-(5**0.5))/2)])   

In [57]:
#Пример:
sections(coolg, f1, -1., 1., 25, 0)

-9.644875678381916e-06

$\textbf{Метод парабол}$

In [47]:
def Par(f, h, x0, k, say):
    def temp(y):
        return y - (h/2)*(f(y + h) - f(y - h))/(f(y + h) + f(y - h) - 2*f(y))
    return CanonicalMPI(temp, x0, k, say)

$\textbf{Метод покоординатного спуска:}$

In [65]:
#Пусть g(f0) - значение, минимизирующее f0 (f0 - ф-я одной переменной)
def coord(g, f, I0, k, say = 0):
    for k in range(k):
        for i in range(len(I0)):
            if(say != 0):
                print(k, "th iteration, ", i, "th coordinate:", I0)
            def tmp(y):
                J = np.array([I0[j]*(1 - KronDel(i,j)) + KronDel(i,j)*y for j in range(len(I0))])
                return f(J)
            I0[i] = g(tmp)
        if(say != 0):
                print(k, "th iteration, ", len(I0), "th coordinate:", I0)
    return I0

In [70]:
#Пример
def G(F):
    return Par(F, 0.01, 1, 100, 0)
def f(I):
    return I[0]**2 + (np.exp(I[1]) - 2)**2
coord(G, f, np.array([2., 3.]), 10, 1)

0 th iteration,  0 th coordinate: [2. 3.]
0 th iteration,  1 th coordinate: [6.31397301e-20 3.00000000e+00]
0 th iteration,  2 th coordinate: [6.31397301e-20 6.93097181e-01]
1 th iteration,  0 th coordinate: [6.31397301e-20 6.93097181e-01]
1 th iteration,  1 th coordinate: [1.08420217e-19 6.93097181e-01]
1 th iteration,  2 th coordinate: [1.08420217e-19 6.93097181e-01]
2 th iteration,  0 th coordinate: [1.08420217e-19 6.93097181e-01]
2 th iteration,  1 th coordinate: [1.08420217e-19 6.93097181e-01]
2 th iteration,  2 th coordinate: [1.08420217e-19 6.93097181e-01]
3 th iteration,  0 th coordinate: [1.08420217e-19 6.93097181e-01]
3 th iteration,  1 th coordinate: [1.08420217e-19 6.93097181e-01]
3 th iteration,  2 th coordinate: [1.08420217e-19 6.93097181e-01]
4 th iteration,  0 th coordinate: [1.08420217e-19 6.93097181e-01]
4 th iteration,  1 th coordinate: [1.08420217e-19 6.93097181e-01]
4 th iteration,  2 th coordinate: [1.08420217e-19 6.93097181e-01]
5 th iteration,  0 th coordinate: 

array([1.08420217e-19, 6.93097181e-01])